## Introduction

1. full problem
1. quasi-periodic boundaries
1. linear problem with curved coordinates
1. linear problem 



## Biot Savart Field $H_{BS}$ of a single Conductor

\begin{align}
    \mathbf{H}_{BS} = \frac{I_0}{2\pi||\mathbf{r}||}\mathbf{e}_{\varphi}
\end{align}

In [31]:
r1, r2, r3, r4, phi_deg, maxh = 0.01, 0.02, 0.03, 0.032, 45, 0.001
maxh = 0.1
maxhEdges = 0.001

order0 = 2

In [32]:
from permanentMagnetGeometry import  simpleGeoMagnetOnIron


import numpy as np
from ngsolve import *
from ngsolve.webgui import Draw
r1, r2, r3, r4, phi_deg, maxh = 0.01, 0.02, 0.03, 0.032, 45, 0.001
maxh = 0.1
maxhEdges = 0.001

cMesh_simple = simpleGeoMagnetOnIron(r1, r2, r3, r4, (r3+r2)/2 * np.pi  * 90/180, (r4+r1)/2 * np.pi, maxh=maxh, maxhEdges=maxhEdges, periodic=False)
#cMesh_simple = simpleGeoMagnetOnIron(r1, r2, r3, r4,  np.pi/2 ,  np.pi, maxh=maxh, maxhEdges=maxhEdges, periodic=False)
mesh_simple = cMesh_simple.mesh

# cMesh_simple = simpleGeoMagnetOnIron(d0 = (r4-r3)*10, Hm=r3-r2, HFe=r2-r1, Bm=np.pi/2, B=np.pi, maxh=maxh, periodic=True, maxhEdges = maxhEdges)
# mesh_simple = cMesh_simple.mesh

print("domains", mesh_simple.GetMaterials())
print("boundaries", set(mesh_simple.GetBoundaries()))


#print(Integrate(1, mesh_periodic))
#print(Integrate(1, mesh_simple))





domains ('rotor', 'magnet', 'air')
boundaries {'left_iron', 'left_air', 'right_air', 'mag_rot', 'outer', 'right_iron', 'interface', 'inner'}


In [33]:
from ngsolve import *
from ngsolve.webgui import Draw

import numpy as np

def getR():
    return y
def getPhi():
    return -x * np.pi/cMesh_simple.B 

def getTrafo():
    r = getR()
    phi = getPhi()
    coef = np.pi/(r*2)
    return CF((coef * sin(phi), -coef * cos(phi), cos(phi), sin(phi)), dims=(2, 2))

def HBS_singleConductor_polar(I0, xy_center, dim=2):
    r = getR()
    phi = getPhi()

    rphi_center = (sqrt(xy_center[0]**2 + xy_center[1]**2), atan2(xy_center[1], xy_center[0])) 

    phi_s = phi - rphi_center[1]
    r_s = r - rphi_center[0]

    if dim == 2:
        e_phi = CF((phi, 0))/(Norm(phi) + 1e-15)
    else:
        e_phi = e_phi = CF((phi, 0, 0))/(Norm(phi)+1e-15)

    return I0/(2*np.pi*r_s + 1e-15) * e_phi
    


def HBS_singleConductor_cartesian(I0, xy_center, dim=2):
    

    # shifted coordninates
    x_s = x - xy_center[0]
    y_s = y - xy_center[1]

    r = sqrt(x_s ** 2 + y_s **2)

    if dim == 2:
        e_phi = CF((y_s, -x_s))/(r+1e-15)
    else:
        e_phi = e_phi = CF((y_s, -x_s, 0))/(r+1e-15)

    return I0/(2*np.pi*r + 1e-15) * e_phi

def animate(val, mesh, ti=np.linspace(0, 2*np.pi, 100), pause=0.1, **kwargs):
    import time
    t = Parameter(ti[0])
    scene = Draw(val * exp(1j*t), mesh, **kwargs)
    
    for tt in ti:
        
        t.Set(tt)
        scene.Redraw()
        print(tt, end="\r")
        time.sleep(pause)


    

In [34]:
## mesh
from permanentMagnetGeometry import  fullGeoMagnet
from ngsolve.webgui import Draw




cMesh = fullGeoMagnet(r1, r2, r3, r4, phi_deg=45, maxh=maxh, maxhEdges=maxhEdges, periodic=False)
mesh = cMesh.mesh


print("domains", mesh.GetMaterials())
print("boundaries", set(mesh.GetBoundaries()))

Draw(mesh)


domains ('air', 'rotor', 'magnet_top', 'magnet_bottom')
boundaries {'mag_rot', 'outer', 'interface', 'inner'}


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [35]:
valid_excitation_types = ("BSF", "surfacecurrent")
excitation_type = "BSF"
# ------------------------------------------------------------------------------
# --- excitation
# ------------------------------------------------------------------------------
f = 50
omega = 2*np.pi * f


if excitation_type not in valid_excitation_types:
    raise ValueError(f"not a valid excitation type use: {valid_excitation_types}")



def getExcitatation(excitation_type, polar_coordinates=False):


    if excitation_type == "BSF":

        if polar_coordinates == True:
            HBS_singleConductor_foo = HBS_singleConductor_polar

        else:
            HBS_singleConductor_foo = HBS_singleConductor_cartesian
        # circle
        phi_i = (0, 2*np.pi/3, 2*np.pi*2/3)        


        points = [(r4 * cos(phi), r4*sin(phi)) for phi in phi_i]
        H_BS_i = [HBS_singleConductor_foo(1 * np.exp(1j * phi), xy) for xy, phi in zip(points, phi_i)]

        points = [(r4 * cos(phi + np.pi), r4*sin(phi+ np.pi)) for phi in phi_i]
        H_BS_i += [HBS_singleConductor_foo(-1 * np.exp(1j * phi), xy) for xy, phi in zip(points, phi_i)]

        HBS = sum(H_BS_i)

        return HBS
    elif excitation_type == "surfacecurrent":

        # surface current
        K0 = 100
        phi = atan2(y, x)

        a = exp(1j * 2*np.pi/3)

        Ki = lambda i : K0 * cos(phi + (i-1) * 2*np.pi/3 ) 
        K1 = Ki(1)
        K2 = Ki(2) * a * a
        K3 = Ki(3) * a  

        K = (K1 + K2 + K3)/1.5

        return K


HBS = getExcitatation(excitation_type)
print("excited by ", excitation_type)

excited by  BSF


In [36]:

# ------------------------------------------------------------------------------
# --- materials
# ------------------------------------------------------------------------------
sigma_Fe = 2e6
sigma_air = 0
sigma_magnet = 556e3

mu_air = 4e-7*np.pi
mu_Fe = 1000 * mu_air
mu_magnet = 1.05 * mu_air



## $\mathbf{A}$ Formulation

\begin{align*}
\int_\Omega \nu\nabla\times\mathbf{A}\cdot\nabla\times\mathbf{A}' + j\omega\sigma\mathbf{A}\cdot\mathbf{A}' \;d\Omega = \int_\Omega\mathbf{H}_{BS}\cdot\nabla\times\mathbf{A}'\;d\Omega
\end{align*}

with homogeneous Dirichlet boundaries on the inner and the outer


In two dimensions -> single component magnetic vector potential
\begin{align*}
\mathbf{A} = u \mathbf{e}_z \text{ and } \mathbf{B} = \nabla\times\mathbf{A} = \partial_y u\mathbf{e}_x - \partial_x u\mathbf{e}_y
\end{align*}
and therefore 

\begin{align*}
\int_{\Omega_{2D}} \nu\nabla u\cdot \nabla u' + j\omega\sigma u\cdot u' \;d\Omega = \int_{\Omega_{2D}}\mathbf{H}_{BS}\cdot (\partial_y u'\mathbf{e}_x - \partial_x u'\mathbf{e}_x)\;d\Omega
\end{align*}


In [37]:


def solveFullWithA(mesh, order0, omega, HBS, periodic=False, polar=False):
    if not polar:
        P = CF((1, 0, 0, 1), dims=(2, 2))
    else:
        r = getR()
        P = CF((-1, 0, 0, 1/r), dims=(2, 2))
    # ------------------------------------------------------------------------------
    # --- materials
    # ------------------------------------------------------------------------------
    mu = mesh.MaterialCF({"air": mu_air, "rotor": mu_Fe, "magnet.*": mu_magnet}, default=0.001)
    nu = 1/mu
    sigma = mesh.MaterialCF({"air": sigma_air, "rotor": sigma_Fe, "magnet.*": sigma_magnet}, default=0.001)
    rho = 1/sigma
    
    # ------------------------------------------------------------------------------
    # --- fem
    # ------------------------------------------------------------------------------
    dir_A = "inner"

    if periodic:
        fes = Periodic(H1(mesh, order=order0, dirichlet=dir_A, complex=True), phase=[-1, -1])
    else:
        fes = H1(mesh, order=order0, dirichlet=dir_A, complex=True)
    

    u, v = fes.TnT()


    sol = GridFunction(fes, "A")
    gradsol = grad(sol)


    # ------------------------------------------------------------------------------
    # --- fields
    # ------------------------------------------------------------------------------
    B = CF((gradsol[1], -gradsol[0]), dims=(2,1))
    H = nu * B
    E = -1j * omega * sol
    J = sigma * E


    # ------------------------------------------------------------------------------
    # --- formulation
    # ------------------------------------------------------------------------------

    fuck = cMesh_simple.B/np.pi
    fuck =np.pi/ cMesh_simple.B
    ah = BilinearForm(fes, symmetric=True)
    if not polar:
        ah += nu * grad(u) * grad(v) * dx
        ah += 1j * omega * sigma * u * v * dx("rotor|magnet.*")
    else:
        ah += getR()*np.pi/cMesh_simple.B * nu  * (grad(u)[1] * grad(v)[1] + 1/(getR() **2) * fuck**2 *grad(u)[0] * grad(v)[0]) * dx
        #ah += getR()*np.pi/cMesh_simple.B * nu  * grad(u) * grad(v) * dx
        ah += getR()*np.pi/cMesh_simple.B * 1j * omega * sigma * u * v * dx("rotor|magnet.*")

    # prec = Preconditioner(ah, type="direct")

    gradv = grad(v)
    f = LinearForm(fes)
    if excitation_type == "BSF":
        if not polar:
            f  += HBS * CF((gradv[1], -gradv[0])) * dx
        else:
            f  += getR()*np.pi/cMesh_simple.B * HBS * CF((-grad(v)[1], 1/getR() * fuck * grad(v)[0])) * dx
    # if excitation_type == "surfacecurrent":
    #     f  += K * v * ds("outer")


    ah.Assemble()
    f.Assemble()


    # BVP(bf = ah, lf= f, pre=prec, gf=sol, maxsteps=10).Do()
    sol.vec.data = ah.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec


    if not polar:
        losses_magnet = .5 * Integrate(InnerProduct(E, J), mesh, definedon=mesh.Materials("magnet.*")).real
        losses_rotor = .5 * Integrate(InnerProduct(E, J), mesh, definedon=mesh.Materials("rotor")).real
    else:
        losses_magnet = .5 * Integrate((getR()*np.pi/cMesh_simple.B) * InnerProduct(E, J), mesh, definedon=mesh.Materials("magnet.*")).real
        losses_rotor = .5 * Integrate((getR()*np.pi/cMesh_simple.B) * InnerProduct(E, J), mesh, definedon=mesh.Materials("rotor")).real

    if periodic:
        losses_magnet *= 2
        losses_rotor *= 2

    return sol, B, E, H, J, losses_magnet, losses_rotor

A, B_A, E_A, H_A, J_A, losses_magnet_A, losses_rotor_A = solveFullWithA(mesh, order0, omega=omega, HBS=HBS)

In [38]:
#animate(J, mesh, ti= np.linspace(0, 4*np.pi, 150), pause=0.01 )
sceneJ_A  = Draw(J_A, mesh, animate=True, max = 500, min = -500)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [39]:
print("excitation type", excitation_type)

excitation type BSF



## $\mathbf{T}$,$\Phi$ - $\Phi$ Formulation

\begin{align}
\int_{\Omega_c} \rho\nabla\times\mathbf{T}\cdot\nabla\times\mathbf{T}' + j\omega\mu(\mathbf{T}-\nabla\Phi)\cdot(\mathbf{T}'-\nabla\Phi') \;d\Omega = -\int_\Omega j\omega\mu(\mathbf{H}_{BS})\cdot(\mathbf{T}'-\nabla\Phi')\;d\Omega
\end{align}

with homogeneous Dirichlet boundaries on the inner and the outer


In two dimensions ->  current vector potential
\begin{align}
\mathbf{J} = \nabla\times\mathbf{T} =\nabla\times(T_x\mathbf{e}_x + T_y\mathbf{e}_y) = (\partial_x T_x + \partial_y T_y)\mathbf{e}_z
\end{align}
and therefore 

\begin{align}
\int_{\Omega_{2D}} \rho\nabla \times \mathbf{T} \cdot \nabla \times \mathbf{T}' + j\omega\mu (\mathbf{T} - \nabla\Phi)\cdot (\mathbf{T}' - \nabla\Phi') \;d\Omega = -\int_\Omega j\omega\mu(\mathbf{H}_{BS})\cdot(\mathbf{T}'-\nabla\Phi')\;d\Omega

\end{align}


In [40]:
def solveFullWithTPhi(mesh, order0, omega, HBS, periodic = False):

    # ------------------------------------------------------------------------------
    # --- materials
    # ------------------------------------------------------------------------------
    mu = mesh.MaterialCF({"air": mu_air, "rotor": mu_Fe, "magnet.*": mu_magnet}, default=0.001)
    nu = 1/mu
    sigma = mesh.MaterialCF({"air": sigma_air, "rotor": sigma_Fe, "magnet.*": sigma_magnet}, default=0.001)
    rho = 1/sigma
    
    # ------------------------------------------------------------------------------
    # --- fem
    # ------------------------------------------------------------------------------

    dir_T  = "interface|inner"
    dir_Phi  = "outer"


    fesT = HCurl(mesh, order=order0, dirichlet=dir_T, complex=True, definedon=mesh.Materials("rotor|magnet.*"))
    fesPhi = H1(mesh, order=order0+1, dirichlet=dir_Phi, complex=True)

    if periodic:
        phase = [-1, -1]
        fes = FESpace([Periodic(fesPhi,phase), Periodic(fesT, phase)])
    else:
        fes = FESpace([fesPhi, fesT])

    trials = fes.TrialFunction()
    tests = fes.TestFunction()


    uPhi, vPhi = trials[0], tests[0]
    uT, vT = trials[1], tests[1]

    sol = GridFunction(fes, "sol")
    Phi = sol.components[0]
    T = sol.components[1]



    # ------------------------------------------------------------------------------
    # --- fields
    # ------------------------------------------------------------------------------
    H = HBS + T - grad(Phi)
    B = mu * H 
    J = curl(T)
    E = rho * J 


    # ------------------------------------------------------------------------------
    # --- formulation
    # ------------------------------------------------------------------------------

    ah = BilinearForm(fes, symmetric=True)
    ah += rho * curl(uT) * curl(vT) * dx("rotor|magnet.*")
    ah += 1j * omega * mu * (uT - grad(uPhi)) * (vT - grad(vPhi)) * dx("rotor|magnet.*")
    ah += 1j * omega * mu * (- grad(uPhi)) * (- grad(vPhi)) * dx("air")

    ah += 1e-3 * uPhi * vPhi * dx("rotor|magnet.*")

    # prec = Preconditioner(ah, type="direct")



    f = LinearForm(fes)
    if excitation_type == "BSF":
        f  += - 1j*omega * mu *  HBS * (vT - grad(vPhi)) * dx
    if excitation_type == "surfacecurrent":
        raise ValueError("surfacecurrent excitation for T Phi not implemented")


    ah.Assemble()
    f.Assemble()


    # BVP(bf = ah, lf= f, pre=prec, gf=sol, maxsteps=10).Do()
    sol.vec.data = ah.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec


    losses_magnet = .5 * Integrate(InnerProduct(E, J), mesh, definedon=mesh.Materials("magnet.*")).real
    losses_rotor = .5 * Integrate(InnerProduct(E, J), mesh, definedon=mesh.Materials("rotor")).real

    if periodic:
        losses_magnet *= 2
        losses_rotor *= 2

    return T, Phi, B, E, H, J, losses_magnet, losses_rotor

T, Phi, B_TPhi, E_TPhi, H_TPhi, J_TPhi, losses_magnet_TPhi, losses_rotor_TPhi = solveFullWithTPhi(mesh, order0, omega, HBS=HBS)

In [41]:
#animate(J, mesh, ti= np.linspace(0, 4*np.pi, 150), pause=0.01 )
sceneJ_TPhi  = Draw(J_TPhi, mesh, animate=True, max = 500, min = -500)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

## Quasi Periodic Boundaries

boundaries in mesh have to be \textit{identified} as well

In [42]:
cMesh_periodic = fullGeoMagnet(r1, r2, r3, r4, phi_deg=45, maxh=maxh, maxhEdges=maxhEdges, periodic=True)
mesh_periodic = cMesh_periodic.mesh



print("domains", mesh_periodic.GetMaterials())
print("boundaries", set(mesh_periodic.GetBoundaries()))

Draw(mesh_periodic)

domains ('air', 'rotor', 'magnet_top')
boundaries {'periodic_rotor', 'mag_rot', 'outer', 'interface', 'inner', 'periodic_air'}


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

### $\mathbf{A}$ Formulation

In [43]:

A, B_A_per, E_A_per, H_A_per, J_A_per, \
    losses_magnet_A_per, losses_rotor_A_per \
    = solveFullWithA(mesh_periodic, order0, omega=omega, HBS=HBS, periodic=True)

In [44]:
sceneJ_APer = Draw(J_A_per, mesh_periodic, max = 500, min = -500)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

## $\mathbf{T}$,$\Phi$ - $\Phi$ Formulation

In [45]:
T_per, Phi_per, B_TPhi_per, E_TPhi_per, \
    H_TPhi_per, J_TPhi_per, \
    losses_magnet_TPhi_per, losses_rotor_TPhi_per \
    = solveFullWithTPhi(mesh_periodic, order0, omega, HBS=HBS, periodic=True)

In [46]:
sceneJ_TPhi  = Draw(J_TPhi_per, mesh_periodic, animate=True, max = 500, min = -500)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

## Straight Model

### Biot Savart field

In [47]:
HBS_polar = getExcitatation(excitation_type, polar_coordinates=True)

#HBS_polar = HBS_singleConductor_polar(1, (0, r2))
HBS_full = getExcitatation(excitation_type, polar_coordinates=False)
Draw(HBS_polar, mesh_simple)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

### $\mathbf{A}$ Formulation

\begin{align*}
\int_\Omega \nu\nabla\times\mathbf{A}\cdot\nabla\times\mathbf{A}' + j\omega\sigma\mathbf{A}\cdot\mathbf{A}' \;d\Omega = \int_\Omega\mathbf{H}_{BS}\cdot\nabla\times\mathbf{A}'\;d\Omega
\end{align*}

with homogeneous Dirichlet boundaries on the inner and the outer


In two dimensions polar coodinates -> single component magnetic vector potential
\begin{align*}
\mathbf{A} = u(\varphi, r) \mathbf{e}_z \text{ and } \mathbf{B} = \nabla\times\mathbf{A} =  -
\partial_r u \mathbf{e}_\varphi + \frac{1}{r}\partial_\varphi u\mathbf{e}_r
\end{align*}
and therefore 

\begin{align*}
\int_{\Omega_{2D}} \nu
\begin{pmatrix}1&0\\0&r^{-2}
\end{pmatrix}
\nabla u\cdot \nabla u' r + j\omega\sigma u\cdot u' r \;d\Omega = \int_{\Omega_{2D}}\widetilde{\mathbf{H}_{BS}}\cdot 
(-\partial_r u' \mathbf{e}_\varphi + \frac{1}{r}\partial_\varphi u'\mathbf{e}_r)r\;d\Omega

\end{align*}

and the transformation of the Biot-Savart field $\widetilde{\mathbf{H}_{BS}}$
\begin{align*}
\widetilde{\mathbf{H}_{BS}} = \begin{pmatrix}r^{-1}\sin(\varphi)& -r^{-1}\cos(\varphi)\\ \cos(\varphi)& \sin(\varphi)\end{pmatrix}\mathbf{H}_{BS}(r\cos(\varphi), r\sin(\varphi))
\end{align*}


In [49]:

print(1/(cMesh_simple.B/np.pi))
#print(Integrate(HBS_full.Norm(), mesh_periodic))
f = CF((x, 0))
f = CF((1, 0))
f2 = CF((getR()*cos(getPhi()), 0))
f2 = CF((1, 0))

f = HBS_full
f2 = getExcitatation(excitation_type, polar_coordinates=True)
a = Integrate(f.Norm(), mesh_periodic, order=10)
b = Integrate((getR()*np.pi/cMesh_simple.B) * (f2).Norm(), mesh_simple, order=10)
print("int_original ", a)
print("int_straight", b)

HBS_polar = getExcitatation(excitation_type, polar_coordinates=True)
print("ratio", a/b)


47.61904761904761
int_original  0.036613831735526695
int_straight 4.9609929882066685e-18
ratio 7380343375321338.0


In [50]:
# transformation of the Biot-Savart field
#Trafo = CF((1/r*sin(phi), -1/r*cos(phi), cos(phi), sin(phi)), dims=(2, 2))


A, B_A_per, E_A_per, H_A_per, J_A_per, \
    losses_magnet_A_per, losses_rotor_A_per \
    = solveFullWithA(mesh_periodic, order0, omega=omega, HBS=HBS, periodic=True, polar=False)

A, B_A_simple, E_A_simple, H_A_simple, J_A_simple, \
    losses_magnet_A_simple, losses_rotor_A_simple \
    = solveFullWithA(mesh_simple, order0, omega=omega, HBS= HBS_polar, periodic=True, polar=True)


print("l,mag", losses_magnet_A_per)
print("l,mag, polar", losses_magnet_A_simple)

print(losses_magnet_A_per/losses_magnet_A_simple)

print("l,rot", losses_rotor_A_per)
print("l,rot, polar", losses_rotor_A_simple)
print(losses_rotor_A_per/losses_rotor_A_simple)


l,mag 3.4586115327030766e-05
l,mag, polar 2.651196366055905e-40
1.3045474778800847e+35
l,rot 1.4595788057063392e-05
l,rot, polar 1.3050862449450012e-39
1.1183772807044249e+34


In [51]:
sceneJ_ASimple = Draw(J_A_simple, mesh_simple)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

## Losses

In [52]:
print(f"Losses in the MAGNET")
print(" \t\tA Formulation\t\t T Phi")

print(f"full \t\t{losses_magnet_A*1e6:0.3f} \t\t\t {losses_magnet_TPhi*1e6:0.3f}")
print(f"periodic \t{losses_magnet_A_per*1e6:0.3f} \t\t\t {losses_magnet_TPhi_per*1e6:0.3f}")
print(f"straight \t{losses_magnet_A_simple*1e6:0.3f} \t\t\t {0}")

print("\n\n")
print(f"Losses in the Rotor")
print(" \t\tA Formulation\t\t T Phi")

print(f"full \t\t{losses_rotor_A*1e6:0.3f} \t\t\t {losses_rotor_TPhi*1e6:0.3f}")
print(f"periodic \t{losses_rotor_A_per*1e6:0.3f} \t\t\t {losses_rotor_TPhi_per*1e6:0.3f}")
print(f"straight \t{losses_rotor_A_simple*1e6:0.3f} \t\t\t {0}")



Losses in the MAGNET
 		A Formulation		 T Phi
full 		34.774 			 34.665
periodic 	34.586 			 34.668
straight 	0.000 			 0



Losses in the Rotor
 		A Formulation		 T Phi
full 		14.593 			 14.601
periodic 	14.596 			 14.603
straight 	0.000 			 0


## PLAYGROUND


\begin{align*}
\begin{pmatrix}x\\y\end{pmatrix}\rightarrow\begin{pmatrix}\varphi\\r\end{pmatrix}\\
x \in [0, \pi]\\
y \in [r_1, r_4]
\end{align*}



In [53]:
P = (r2, 0)
field = -CF((P[0]-x, P[1]-y))

#Draw(field, mesh, vectors=True)

if False:
    r = y +r2
    phi = (x * np.pi/cMesh_simple.B + np.pi/2)
    field_new = -CF((P[0]-r*cos(phi), P[1]-r*sin(phi)))
    A = CF((-sin(phi), cos(phi), cos(phi), sin(phi)), dims=(2, 2))

elif False:
    r = y +r2
    phi = x * np.pi/cMesh_simple.B + np.pi/2+np.pi
    field_new = -CF((P[0]-r*cos(phi), P[1]-r*sin(phi)))
    #A = CF((sin(phi), -cos(phi), -cos(phi), -sin(phi)), dims=(2, 2))
    A = CF((-sin(phi), cos(phi), cos(phi), sin(phi)), dims=(2, 2))
else:
    r = y +r2
    phi = -x * np.pi/cMesh_simple.B + np.pi/2
    field_new = -CF((P[0]-r*cos(phi), P[1]-r*sin(phi)))
    #A = CF((sin(phi), -cos(phi), -cos(phi), -sin(phi)), dims=(2, 2))
    A = CF((1/r*sin(phi), -1/r*cos(phi), cos(phi), sin(phi)), dims=(2, 2))



#Draw(phi, mesh_simple, vectors=True)
#Draw(r, mesh_simple, vectors=True)
Draw(field, mesh_periodic, vectors=True)
Draw(A * field_new, mesh_simple, vectors=True)



WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [54]:
a = CF((1, 0, 0, 1), dims=(2, 2))
Det(a)

In [55]:
from netgen.occ import *
import numpy as np
from ngsolve import *
from ngsolve.webgui import Draw
maxh = 1/10

wp = WorkPlane()
wp.MoveTo(0, 0.1)
rec = wp.Rectangle(np.pi, 0.9).Face()
geo = Glue([rec])
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=maxh))

wp = WorkPlane()
wp.MoveTo(-2, 0)
rec = wp.Rectangle(4, 2).Face()
wp.MoveTo(0, 0)
outer = wp.Circle(1).Face()
inner = wp.Circle(0.1).Face()
geo = Glue([rec*(outer-inner)])
mesh2 = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=maxh))
mesh2.Curve(3)

r = y 
phi = x


trafo_toxy = CF((r*cos(phi), r*sin(phi)), dims=(2,1))
fes_toxy = VectorH1(mesh, order=2)
gfu_toxy = GridFunction(fes_toxy)
gfu_toxy.Set(trafo_toxy)
#J_toxy = grad(gfu_toxy).trans
#J_toxy = CF((cos(phi), -r*sin(phi), sin(phi), r*cos(phi)), dims=(2,2))
#J_toxy = CF((-sin(phi), cos(phi), cos(phi), sin(phi)), dims=(2,2))
#J_toxy = CF((cos(phi), sin(phi), -sin(phi), cos(phi)), dims=(2,2)).trans
J_toxy = CF((trafo_toxy.Diff(r), trafo_toxy.Diff(phi)), dims=(2,2)).trans


trafo_torphi = CF((sqrt(x**2+y**2), atan2(y, x)), dims=(2,1))
fes_torphi = VectorH1(mesh, order=2)
gfu_torphi = GridFunction(fes_torphi)
gfu_torphi.Set(trafo_torphi)
J_torphi = grad(gfu_torphi).trans
J_torphi = CF((trafo_torphi.Diff(x), trafo_torphi.Diff(y)), dims=(2,2)).trans


HBSxy = lambda xy :  1/(sqrt((x-xy[0])**2+(y-xy[1])**2 + 1e-12)**2) * CF((y-xy[1], -x-xy[0]))

In [56]:
#J_toxy = CF((trafo_toxy.Diff(r), trafo_toxy.Diff(phi)), dims=(2,2)).trans
print(Integrate(1, mesh2))
print(Integrate(Det(J_toxy) * 1, mesh))

print(Integrate(1, mesh, order=10))
print(Integrate(Det(J_torphi) * 1, mesh2, order=10))

# Draw(CF((r*cos(phi), r*sin(phi)))[1], mesh, deformation=trafo_toxy*1e6)
# Draw(CF((x, y)) [1], mesh2)



HBS_polar = getExcitatation(excitation_type, polar_coordinates=True)

#HBS_polar = HBS_singleConductor_polar(1, (0, r2))
HBS_full = getExcitatation(excitation_type, polar_coordinates=False)

Draw(HBS_polar, mesh, deformation=trafo_toxy*1e6)
Draw(HBS_full, mesh2)

1.5550866575089293
1.5550883635269412
2.827433388230791
2.8274136788331994


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
Draw(J_toxy/Det(J_toxy) * CF((0, -phi))/Norm(phi + 1e-13), mesh, vectors=True, deformation=trafo_toxy * 1e6)
Draw( CF((y, -x))/sqrt(x**2+y**2), mesh2, vectors=True)

#Draw( HBSxy((0,0)), mesh2, vectors=True)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:



J = CF((cos(phi), -r*sin(phi), sin(phi), r*cos(phi)), dims=(2,2))


Cof = (Inv(J)).trans

Cof = Det(J) * Inv(J)

print(Integrate(Det(J), mesh))
print(Integrate(1, mesh2))

print(Integrate(Det(J) * r * sin(phi), mesh, order=10))
print(Integrate(y, mesh2, order=10))

print(Integrate(Det(J)*r, mesh))
print(Integrate(sqrt(x**2+y**2), mesh2))



print(Integrate((Cof * CF((r*sin(phi), 0))).Norm(), mesh))
print(Integrate((CF((y, 0))).Norm(), mesh2))




1.5550883635269412
1.5550866575089293
0.6659999999999947
0.6660000590246437
1.0461503536453978
1.04615044864557
0.896257285749054
0.6660000590246464


In [ ]:
Draw(CF((x, 0)), mesh, vectors=True, deformation=deform)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene